In [1]:
import http.cookiejar
import json
import os
import re

import requests
from bs4 import BeautifulSoup

In [2]:
def load_cookies_json(path):

    cookie_jar = http.cookiejar.MozillaCookieJar()
    with open(path) as f:
        file = json.loads(f.read())

    for cookie in file:

        if cookie.get("expirationDate") != None:
            exp = str(cookie.get("expirationDate", ""))

        elif cookie.get("expiry") != None:
            exp = str(cookie.get("expiry", ""))

        c = http.cookiejar.Cookie(
            version=0,
            name=cookie.get("name", ""),
            value=cookie.get("value", ""),
            port=None,
            port_specified=False,
            domain=cookie.get("domain", ""),
            domain_specified=True,
            domain_initial_dot=False,
            path=cookie.get("path", ""),
            path_specified=True,
            secure=False,
            expires=exp,
            discard=False,
            comment=None,
            comment_url=None,
            rest="",
        )

        cookie_jar.set_cookie(c)

    return cookie_jar

In [3]:
def searcher(link):
    header = {
        "Accept-Language": "en-US,en;q=0.5",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; rv:78.0) Gecko/20100101 Firefox/78.0",
    }
    session.headers = header
    data = session.get(link)
    soup = BeautifulSoup(data.content, "html.parser")
    return soup

# Load my FB cookie

In [4]:
session = requests.Session()
session.cookies = load_cookies_json("../cookie.json")

# Extract Username, Userid, Name

In [5]:
response = searcher("https://www.facebook.com/profile.php")
data = {}

t = re.compile(r"USER_ID.+?SHORT_NAME").search(str(response)).group(0)
USER_ID, NAME = t.replace('"', "").replace(",SHORT_NAME", "").split(",")
USER_ID = USER_ID.split(":")[-1]
NAME = NAME.split(":")[-1]

data["username"] = (
    re.search("userVanity.+?,", str(response))
    .group(0)
    .replace('"', "")
    .replace(",", "")
    .split(":")[-1]
)
data["uid"] = USER_ID
data["name"] = NAME
print(data)

{'username': 'SWEETERROR404', 'uid': '100004486669038', 'name': 'Mubeen Ahmad'}


# see more links

In [6]:
link = [f"https://mbasic.facebook.com/{data['username']}/friends"]
for i in link:
    response = searcher(i)
    try:
        res = response.find("div", id="m_more_friends").find("a").get("href")
        link.append(f"https://mbasic.facebook.com{res}")
    except:
        print("Link Complete")

Link Complete


# profile links

In [7]:
import concurrent.futures
profile_links = set()

def profile(link):
    response = searcher(link)
    href = ["https://mbasic.facebook.com" + i.get("href") for i in response.find_all("a", class_="_5pxc")]
    profile_links.update(href)

def process_links(links):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(profile, links)

process_links(link)

In [9]:
print('total friends',len(profile_links))

total friends 1497


# deactivate accounts

In [10]:
deactivate_accounts = [i for i in profile_links if i.find('friends/hovercard') != -1]
print('total deactivate friends',len(deactivate_accounts))

total deactivate friends 20


# unfriend create post requests

In [12]:
final = {}

count = 1 # just for counter
def unfriend(pl):
    global count
    
    params = {}
    dta = {}
    
    res = searcher(pl)

    # Extract removefriend link
    l1 = "https://mbasic.facebook.com" + re.search('/removefriend.php.+?"', str(res)).group(0).replace('"', "").replace("amp;", "")
    res = searcher(l1)  # Searcher 2

    # Regular expression patterns
    fb_dtsg_pattern = re.compile('fb_dtsg.+?"/><div')
    name_pattern = re.compile("Are you.+?</a>")
    par_pattern = re.compile('/a/friends/remove/.+?"')

    # Extract fb_dtsg, jazoest, confirm
    raw = fb_dtsg_pattern.search(str(res)).group(0)
    dta["fb_dtsg"], dta["jazoest"] = re.findall('value="([^"]+)"', raw)[:2]
    dta["confirm"] = "Confirm"

    # Extract name
    n = name_pattern.search(str(res)).group(0).replace("</a>", "")[::-1]
    name = n.split(">", 1)[0][::-1]

    # Extract params
    par = (par_pattern.search(str(res))
            .group(0)
            .replace("amp;", "")
            .replace("/a/friends/remove/?", "")
            .replace('"', "")
            .split("&")
        )
    params = dict(kv.split("=") for kv in par)

    print(f"{name} : {params.get('subject_id')} --> {len(profile_links) - count} are left")
    final[params.get('subject_id')] = {'params':params,'data':dta}
    count += 1
    

def process_profiles(profile_links):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(unfriend, profile_links)

process_profiles(profile_links)

# here is all post request ready

In [13]:
len(final)

1497

# for example

In [32]:
par = final['61552822920724']['params']
dt = final['61552822920724']['data']
# session.post('https://mbasic.facebook.com/a/friends/remove/', params=par,  data=dt)

{'subject_id': '61552822920724', 'unfriend_ref': 'inactive_friend_dialog', 'eav': 'AfbyeGmOaKMZzxcEOJ3d4iiSa2pOMyTThhDheha5DcOhO6IfIpl58t4aDJ2SK5SrzW0', 'paipv': '0', 'ext': '1718048577', 'hash': 'AeQRS3RVNW7N1Xf1hUI'}
{'fb_dtsg': 'NAcOjplku1g9GIXc2n4zFyvPPSYlbgJ7Kdbk9GDEGJRJMj7EsOuFoOA:3:1707929477', 'jazoest': '25413', 'confirm': 'Confirm'}


# but i unfriend only deactive friends

In [41]:
final = {}
count = 1 # just for counter
def unfriend(pl):
    global count
    
    params = {}
    dta = {}
    
    res = searcher(pl)

    # Extract removefriend link
    l1 = "https://mbasic.facebook.com" + re.search('/removefriend.php.+?"', str(res)).group(0).replace('"', "").replace("amp;", "")
    res = searcher(l1)  # Searcher 2

    # Regular expression patterns
    fb_dtsg_pattern = re.compile('fb_dtsg.+?"/><div')
    name_pattern = re.compile("Are you.+?</a>")
    par_pattern = re.compile('/a/friends/remove/.+?"')

    # Extract fb_dtsg, jazoest, confirm
    raw = fb_dtsg_pattern.search(str(res)).group(0)
    dta["fb_dtsg"], dta["jazoest"] = re.findall('value="([^"]+)"', raw)[:2]
    dta["confirm"] = "Confirm"

    # Extract name
    n = name_pattern.search(str(res)).group(0).replace("</a>", "")[::-1]
    name = n.split(">", 1)[0][::-1]

    # Extract params
    par = (par_pattern.search(str(res))
            .group(0)
            .replace("amp;", "")
            .replace("/a/friends/remove/?", "")
            .replace('"', "")
            .split("&")
        )
    params = dict(kv.split("=") for kv in par)

    print(f"{name} : {params.get('subject_id')} --> {len(deactivate_accounts) - count} are left")
    final[params.get('subject_id')] = {'params':params,'data':dta}
    count += 1
    

def process_profiles(profile_links):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(unfriend, profile_links)

process_profiles(deactivate_accounts)

Cancel : 61552822920724 --> 19 are left
Cancel : 61554321390852 --> 18 are left
Cancel : 61558665627295 --> 17 are left
Cancel : 100095717671684 --> 16 are left
Cancel : 100007030839105 --> 15 are left
Cancel : 100014269334769 --> 14 are left
Cancel : 61558800553567 --> 13 are left
Cancel : 100069232009285 --> 12 are left
Cancel : 100009718570609 --> 11 are left
Cancel : 61558462177402 --> 10 are left
Cancel : 100070632870315 --> 9 are left
Cancel : 61558869819941 --> 8 are left
Cancel : 100002759137104 --> 7 are left
Cancel : 61557316062619 --> 6 are left
Cancel : 61556766450515 --> 5 are left
Cancel : 100077528901487 --> 4 are left
Cancel : 61559172384092 --> 3 are left
Cancel : 61550051677803 --> 2 are left
Cancel : 61551468543393 --> 1 are left
Cancel : 61554193546536 --> 0 are left


In [43]:
len(final)

20

# unfriends deactivate friends

In [60]:
for i in list(final.values()):
    print(i['params'])
    print(i['data'])
    resp = session.post('https://mbasic.facebook.com/a/friends/remove/', params=i['params'],  data=i['data'])
    print(resp)

{'subject_id': '61552822920724', 'unfriend_ref': 'inactive_friend_dialog', 'eav': 'AfYtxtcUlPdOlMch_0ivmS0CE-YW-JH-sNeWko83Xg_uODB0rKVr2xmxs_YcEazMyOg', 'paipv': '0', 'ext': '1718049350', 'hash': 'AeTff3aIJh6m7iVYnmQ'}
{'fb_dtsg': 'NAcMiaxII_7bJfLBjMYHwmKWqOx1tXjKk6ngfr0cJH_y9U7OMdrLCPQ:3:1707929477', 'jazoest': '25503', 'confirm': 'Confirm'}
<Response [200]>
{'subject_id': '61554321390852', 'unfriend_ref': 'inactive_friend_dialog', 'eav': 'AfZX8OTp7DPZ7c64AQZxxCfJYHkZod4wbtm-mPmUYPSCOuIUYD1hJ_AaKikyOiVCVFo', 'paipv': '0', 'ext': '1718049350', 'hash': 'AeSjH_0EH2o6UIV1YB8'}
{'fb_dtsg': 'NAcNZRfS_oJ_yP-4AqNSxuyIg331TFfY1DqL0YCwhSsHjPYxCEttSIw:3:1707929477', 'jazoest': '25460', 'confirm': 'Confirm'}
<Response [200]>
{'subject_id': '61558665627295', 'unfriend_ref': 'inactive_friend_dialog', 'eav': 'AfaYsllVyLDg2OpzG1p7rblxm7hPmmm_jlWE29Z-9Oaj3v8jiikd9cBfbWzpys3mohQ', 'paipv': '0', 'ext': '1718049350', 'hash': 'AeSP4oWI0DRD6WBRLHI'}
{'fb_dtsg': 'NAcO8y89hj_L3iTJvzjLgw2SgKKION7WCdYCrCCXZ32W

# all deactivate profiles are un friended